# SVI

In [1]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
import matplotlib.pyplot as plt # type: ignore
from scipy.integrate import quad # type: ignore
%matplotlib inline

from svi import *

# real data
vol = pd.read_csv("./2023.csv").filter(["QUOTE_DATE", "EXPIRE_DATE", "DTE", "Log-Moneyness", "P_IV"])  # Select cols
vol = vol.dropna()

vol = vol[vol["DTE"] == 1]  # Subset rows where period 
vol["tau"] = vol["DTE"] / 365  # Creates a new column named tau
vol.rename(columns={"Log-Moneyness": "k"}, inplace=True)  # rename column to k
vol["w"] = vol["P_IV"]**2 * vol["tau"] # total variance

# vol = vol.sort_values(by='k', ascending=True)

log_moneyness = vol["k"]
total_variance = vol["w"]
implied_volatiliy = vol["P_IV"]
time_to_maturity = vol["tau"]

vol

,QUOTE_DATE,EXPIRE_DATE,DTE,k,P_IV,tau,w
144,2023-01-04,2023-01-05,1.0,1.348953,5.41271,0.00274,0.080267
145,2023-01-04,2023-01-05,1.0,1.166632,4.69153,0.00274,0.060303
146,2023-01-04,2023-01-05,1.0,1.012481,4.08850,0.00274,0.045797
147,2023-01-04,2023-01-05,1.0,0.878950,3.56491,0.00274,0.034818
148,2023-01-04,2023-01-05,1.0,0.761167,3.10337,0.00274,0.026386
...,...,...,...,...,...,...,...
1435973,2023-09-28,2023-09-29,1.0,-0.024325,0.19960,0.00274,0.000109
1435975,2023-09-28,2023-09-29,1.0,-0.026593,0.21104,0.00274,0.000122
1435978,2023-09-28,2023-09-29,1.0,-0.029985,0.23167,0.00274,0.000147
1435979,2023-09-28,2023-09-29,1.0,-0.031113,0.24061,0.00274,0.000159


# Calibration of a slice

## Let's make two direct calibrations (no quasi-explicit reparametrization). One with brute force global optimization and a refinement with a simplex algorith, and the second through a differential evolution - DE, optimization refined by a L-BFGS-B method.

In [2]:
weight = 1.
grid = 10

# Fits a slice through direct brute force
pbrute = svi_fit_direct(log_moneyness, total_variance, weight, "brute", grid)
ivbrute = np.sqrt(raw_svi(pbrute[0], log_moneyness) / time_to_maturity)

# Fits a slice through direct Differential Evolution - DE
pDE = svi_fit_direct(log_moneyness, total_variance, weight, "DE", grid)
ivDE = np.sqrt(raw_svi(pDE.x, log_moneyness) / time_to_maturity)

#  Data frame comparison
parameters = ["a", "b", "rho", "m", "sigma", "Obj. Value: wrmse"]

table = pd.DataFrame({"Pars": parameters,
                      "brute": np.append(pbrute[0], pbrute[1]),
                      "DE": np.append(pDE.x, pDE.fun)})

table[["Pars", "brute", "DE"]]

print(f'parameter from brute: {pbrute[0]}')
print(f'parameter from DE: {pDE.x}')

table


parameter from brute: [-0.06581837  0.08850948  0.46292256  0.44929025  0.84348476]
parameter from DE: [1.14026519e-06 4.25782318e-02 9.99994248e-01 4.44856861e-01
 1.39496309e-01]


,Pars,brute,DE
0,a,-6.581837e-02,0.000001
1,b,8.850948e-02,0.042578
2,rho,4.629226e-01,0.999994
3,m,4.492902e-01,0.444857
4,sigma,8.434848e-01,0.139496
5,Obj. Value: wrmse,9.853690e-07,0.000003


In [3]:
g_value_brute = gfun(pbrute[0], log_moneyness)
g_value_DE = gfun(pDE.x, log_moneyness)

print(f'g_value_brute:{g_value_brute.describe()}')
print('\n')
print(f'g_value_DE: {g_value_DE.describe()}')


g_value_brute:count    21694.000000
mean         0.521820
std          0.389252
min         -0.012892
25%          0.097585
50%          0.539105
75%          0.923277
max          1.040173
Name: k, dtype: float64


g_value_DE: count    21694.000000
mean         0.769073
std          0.309412
min         -0.044913
25%          0.648711
50%          0.871043
75%          0.978729
max          1.638499
Name: k, dtype: float64
